In [ ]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import pypsa
import yaml

### Scenario selection

In [ ]:
PREFIX = "/mnt/c/Users/scl38887/Documents/git/pypsa-nz/"

In [ ]:
OUTPUT = PREFIX + "results/figures/comp/"

In [ ]:
# Specify the scenario name and the path here
scenarios = {#"0": PREFIX + f"results/nz_2030_exp/postnetworks/elec_s_10_ec_lc3.0_CO2L_3H_2030_0.071_AB_0export.nc",
            "nz_endo": PREFIX + f"results/nz_endo/postnetworks/elec_s_10_ec_lc3.0_CO2L_3H_2030_0.071_AB_0export.nc",
            #"40": PREFIX + f"results/nz_2030_exp/postnetworks/elec_s_10_ec_lc3.0_CO2L_3H_2030_0.071_AB_40export.nc",
            "nz_v04": PREFIX + f"results/nz_v04/postnetworks/elec_s_10_ec_lc3.0_CO2L_3H_2030_0.071_AB_20export.nc",
}


# Transport scenarios; add full path in order to work
# scenarios = {"10": "nz_nosmr",
#         "15": "nz_ts15",
#         "25": "nz_ts25",
#        }

# 2030-2050 scenarios; add full path in order to work
# scenarios = {"2030": "nz_nosmr",
#         "2050": "nz_2050_cap",
#        }

### Settings

In [ ]:
tech_colors = PREFIX + "config/tech_colors.yaml"

In [ ]:
threshold_dispatch = 1e6 # 4 TWh

In [ ]:
xlabel = "" #"EV share in %" for transport scenarios
plotname = "endogenousexport" #"ev_share" for transport scenarios; "2030-2050cap" for 2030-2050 scenarios

### Data preparation

In [ ]:
def load_config(config):
    with open(config, 'r') as stream:
        try:
            config = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return config

### Dispatch data preparation

In [ ]:
for sc in scenarios:
    print(sc)
    print(scenarios[sc])

In [ ]:
def get_dispatch(threshold_dispatch, show_demand=True):
    """Get dispatch
    """

    dispatch_all = pd.DataFrame()

    for sc in scenarios:

        n = pypsa.Network(scenarios[sc])

        dispatch = n.statistics.dispatch(bus_carrier="AC")[n.statistics.dispatch(bus_carrier="AC") > 0]
        threshold_dispatch_sum = dispatch[dispatch < threshold_dispatch].sum()
        dispatch = dispatch[dispatch > threshold_dispatch]
        dispatch = pd.DataFrame(dispatch.droplevel(0)).T
        dispatch[f"(Dispatch < {threshold_dispatch/1e6} TWh thres.)"] = threshold_dispatch_sum

        dispatch.index = [sc]

        dispatch_all = pd.concat([dispatch_all, dispatch], axis=0) #, ignore_index=True)

    if show_demand == "bar":
        dispatch_demand = n.statistics.dispatch(bus_carrier="AC")[n.statistics.dispatch(bus_carrier="AC") < 0]
        ely_electricity = dispatch_demand.droplevel(0).loc["H2 Electrolysis"].sum() * (-1)
        total_demand = dispatch_demand.sum() * (-1)
        other_demand = total_demand - ely_electricity
        dispatch_all.loc["Demand", "H2 Electrolysis electricity"] = ely_electricity
        dispatch_all.loc["Demand", "Domestic electricity"] = other_demand
        
    elif show_demand == "line":
        dispatch_demand = n.statistics.dispatch(bus_carrier="AC")[n.statistics.dispatch(bus_carrier="AC") < 0]
        ely_electricity = dispatch_demand.droplevel(0).loc["H2 Electrolysis"].sum() * (-1)
        total_demand = dispatch_demand.sum() * (-1)
        other_demand = total_demand - ely_electricity
        dispatch_all.loc[:, "H2 Electrolysis electricity"] = ely_electricity
        dispatch_all.loc[:, "Domestic electricity"] = other_demand
        
    
    return dispatch_all

In [ ]:
dispatch = get_dispatch(threshold_dispatch, show_demand=False)

In [ ]:
dispatch

### Plot data

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

# Get colors from yaml defined in tech_colors
colors = load_config(tech_colors)["tech_colors"]

# Sort columns by highest value in first row
dispatch_sorted = dispatch[dispatch.iloc[-1].sort_values(ascending=False).index]
dispatch_twh = dispatch_sorted / 1e6 # convert to TWh

dispatch_twh.plot.bar(ax=ax, stacked=True, alpha=0.8, color=colors)

plt.xlabel(xlabel)
plt.ylabel("Dispatch in TWh")
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), loc='upper left', bbox_to_anchor=(1, 1))
plt.xticks(rotation=0)
#plt.ylim(0, 250)

plt.grid(axis='y', alpha=0.5) 

fig.savefig(OUTPUT + plotname + f".pdf", bbox_inches="tight")
fig.savefig(OUTPUT + plotname + f".png", bbox_inches="tight")